In [ ]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
import torch.nn as nn
from torchvision import datasets, transforms
import torchvision
import torch.optim as optim
from torchvision.datasets import DatasetFolder
from torch.utils.data import Dataset, DataLoader

from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Loading the Data**

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/MODELS_DATA/Ja/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/MODELS_DATA/Ja/test.csv')

In [ ]:
test_df = test_df.drop('id', axis=1)

In [ ]:
train_df

,labels,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,0,-2.033875,0.978446,-0.142131,-0.177117,-1.470684,1.669562,-0.196530,-0.125239,-0.452284,...,-1.111266,0.716084,0.060039,0.301279,-1.174846,-1.076498,-0.069452,-0.604012,-2.179176,0.558003
1,1,-0.348835,0.294815,-0.557577,-2.020773,-1.234715,1.633930,-1.680658,-0.358146,0.166122,...,0.735240,0.829781,1.521941,1.347946,0.754505,1.330642,-0.754453,0.582956,0.252671,1.495870
2,1,0.113248,-0.607726,-0.947791,0.830851,0.998291,0.498321,-1.493958,0.789572,-1.311018,...,0.104698,0.616189,-1.035953,2.111387,-0.984415,1.148076,-1.433554,0.243372,0.170083,1.274795
3,0,1.223321,-0.479048,-1.925789,1.680377,0.021840,-1.453307,0.605559,-0.019024,1.065448,...,0.360237,-1.957863,-0.123384,1.505329,0.660290,-1.769443,-0.547756,-0.568122,0.244645,0.982116
4,0,0.160109,0.422684,-0.308029,0.227744,0.432854,0.608348,0.193832,1.035091,-0.538868,...,0.416629,1.441766,0.212572,-0.994721,1.143999,-2.166923,-1.199248,-1.028636,0.752791,0.317169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245,0,1.157565,-0.142219,1.043992,1.144946,1.195423,0.248978,-1.505100,-0.874137,-1.782724,...,1.195423,-0.255793,-0.154838,0.413029,-0.482939,-1.277953,-0.445082,1.195423,-0.924614,-0.432462
5246,0,1.424709,0.235910,1.356778,1.368099,-0.318862,1.039765,-0.986854,-0.330184,-1.383120,...,1.424709,-1.066107,0.881258,-0.488691,-1.281223,-1.213291,0.122692,1.175627,-1.145360,0.451026
5247,1,-0.375687,1.524455,0.012514,-0.007917,0.073809,-0.906909,-1.254247,1.606182,0.298557,...,-0.028349,-0.968204,-1.233815,1.626613,-0.191802,1.115823,0.380284,-0.293960,0.135104,1.381434
5248,1,-0.478238,1.666142,0.049609,-0.428752,-0.362771,1.798104,-0.214314,0.775400,-0.379267,...,-0.428752,-1.121552,-0.379267,-0.593705,0.049609,1.765114,0.313533,-0.329781,-1.220524,0.033114


In [ ]:
test_df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,-3.388242,0.868285,-0.427619,-0.678964,-1.625735,0.262761,1.243040,1.537751,-0.352028,-0.114245,...,-0.776403,-0.662884,-0.257091,-1.168413,0.223260,-0.482520,-0.085453,-0.382265,-0.539349,-1.682404
1,-0.496920,0.952381,0.989040,0.451422,0.513516,-0.099658,-1.124326,0.729430,-0.216224,-0.000680,...,0.379635,-1.760084,1.125450,-0.328047,-0.880305,-1.257607,0.964312,2.021104,0.655021,-0.423029
2,1.128369,-0.537951,2.544358,1.165254,-1.904994,0.776961,-0.495768,0.060111,-1.418468,-0.086128,...,1.165254,-1.373589,-0.483701,-0.964782,-0.869555,0.066040,-0.444567,-0.531935,-0.878660,1.099488
3,0.051253,1.746814,0.681177,1.844524,-0.327977,1.226839,-0.085519,0.379008,-1.003667,0.570237,...,-0.442288,-2.794472,-0.763468,-0.789832,-0.113209,-2.703150,-2.058728,1.070627,-0.458045,-0.435825
4,1.423209,-0.983594,-1.694170,1.197507,1.044211,0.518777,-0.298612,-0.365174,0.738447,0.054435,...,-2.624450,-3.200223,0.711422,-0.190394,0.337224,-1.656639,0.707360,-0.562290,1.471181,-0.192000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,0.889888,-0.319077,0.849589,0.822723,0.876455,0.325704,0.876455,-0.910127,0.889888,0.204808,...,0.889888,-2.226556,-0.090717,-1.393713,-0.896694,-0.399675,-0.856395,0.876455,0.863022,-0.601169
2246,1.005737,-0.064755,1.163494,1.163494,1.163494,0.724028,0.712760,-0.785929,-1.225394,1.107152,...,1.163494,-1.270468,-0.932417,-1.169053,-0.008414,-0.605636,-0.323927,1.163494,-1.315541,0.047928
2247,1.252086,1.223561,0.153859,-0.987156,0.239435,-0.003031,-1.158309,1.237823,-1.272410,0.767154,...,0.581740,-1.386512,0.809943,-1.243885,0.153859,-0.630589,1.594391,1.252086,-1.429300,1.408976
2248,1.042624,-0.129166,1.066538,1.030667,1.162195,0.707827,-1.396612,0.014319,-1.025944,0.982839,...,1.078495,-1.193343,0.086061,-0.081338,-0.978116,-0.368307,-0.129166,1.090452,-1.444440,0.468686


# **Scaling the Data**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
features = train_df.iloc[:, 1:]  # Exclude the 'labels' column
labels = train_df['labels']
scaled_features = scaler.fit_transform(features)
train_df_scaled = pd.DataFrame(scaled_features, columns=features.columns)
train_df_scaled['labels'] = labels

In [ ]:
features

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,-2.033875,0.978446,-0.142131,-0.177117,-1.470684,1.669562,-0.196530,-0.125239,-0.452284,-0.128052,...,-1.111266,0.716084,0.060039,0.301279,-1.174846,-1.076498,-0.069452,-0.604012,-2.179176,0.558003
1,-0.348835,0.294815,-0.557577,-2.020773,-1.234715,1.633930,-1.680658,-0.358146,0.166122,-1.656990,...,0.735240,0.829781,1.521941,1.347946,0.754505,1.330642,-0.754453,0.582956,0.252671,1.495870
2,0.113248,-0.607726,-0.947791,0.830851,0.998291,0.498321,-1.493958,0.789572,-1.311018,0.848524,...,0.104698,0.616189,-1.035953,2.111387,-0.984415,1.148076,-1.433554,0.243372,0.170083,1.274795
3,1.223321,-0.479048,-1.925789,1.680377,0.021840,-1.453307,0.605559,-0.019024,1.065448,0.717341,...,0.360237,-1.957863,-0.123384,1.505329,0.660290,-1.769443,-0.547756,-0.568122,0.244645,0.982116
4,0.160109,0.422684,-0.308029,0.227744,0.432854,0.608348,0.193832,1.035091,-0.538868,0.778445,...,0.416629,1.441766,0.212572,-0.994721,1.143999,-2.166923,-1.199248,-1.028636,0.752791,0.317169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245,1.157565,-0.142219,1.043992,1.144946,1.195423,0.248978,-1.505100,-0.874137,-1.782724,0.261597,...,1.195423,-0.255793,-0.154838,0.413029,-0.482939,-1.277953,-0.445082,1.195423,-0.924614,-0.432462
5246,1.424709,0.235910,1.356778,1.368099,-0.318862,1.039765,-0.986854,-0.330184,-1.383120,1.243559,...,1.424709,-1.066107,0.881258,-0.488691,-1.281223,-1.213291,0.122692,1.175627,-1.145360,0.451026
5247,-0.375687,1.524455,0.012514,-0.007917,0.073809,-0.906909,-1.254247,1.606182,0.298557,0.053378,...,-0.028349,-0.968204,-1.233815,1.626613,-0.191802,1.115823,0.380284,-0.293960,0.135104,1.381434
5248,-0.478238,1.666142,0.049609,-0.428752,-0.362771,1.798104,-0.214314,0.775400,-0.379267,0.725914,...,-0.428752,-1.121552,-0.379267,-0.593705,0.049609,1.765114,0.313533,-0.329781,-1.220524,0.033114


In [ ]:
test_df_scaled = scaler.transform(test_df)
test_df_scaled =pd.DataFrame(test_df_scaled, columns=test_df.columns)

In [ ]:
train_df_scaled

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199,labels
0,-2.604266,0.991972,-0.473765,-0.605116,-1.614553,1.671046,-0.022425,-0.055823,-0.325416,-0.418619,...,1.063572,-0.032807,0.583405,-0.999795,-0.902615,0.029085,-1.033815,-2.009790,0.614850,0
1,-0.790418,0.231447,-0.902375,-2.536834,-1.370918,1.631245,-1.504373,-0.312871,0.310758,-2.179461,...,1.171431,1.491790,1.640938,0.851902,1.458978,-0.720660,0.223115,0.488134,1.660502,1
2,-0.293013,-0.772610,-1.304954,0.450997,0.934638,0.362784,-1.317948,0.953811,-1.208823,0.706079,...,0.968807,-1.175800,2.412305,-0.817028,1.279866,-1.463948,-0.136485,0.403303,1.414020,1
3,0.901916,-0.629459,-2.313940,1.341100,-0.073537,-1.817159,0.778487,0.061402,1.235924,0.554999,...,-1.473064,-0.224096,1.799955,0.761479,-1.582447,-0.494427,-0.995809,0.479890,1.087704,0
4,-0.242570,0.373698,-0.644921,-0.180917,0.350830,0.485683,0.367364,1.224779,-0.414487,0.625372,...,1.751990,0.126268,-0.726049,1.225719,-1.972406,-1.207496,-1.483468,1.001844,0.346338,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245,0.831134,-0.254744,0.749941,0.780095,1.138175,0.084272,-1.329073,-0.882347,-1.694082,0.030130,...,0.141603,-0.256899,0.696315,-0.335737,-1.100258,-0.382048,0.871682,-0.721139,-0.489446,0
5246,1.118699,0.165917,1.072637,1.013907,-0.425309,0.967570,-0.811588,-0.282011,-1.282997,1.161031,...,-0.627101,0.823631,-0.214765,-1.101890,-1.036819,0.239390,0.850719,-0.947884,0.495578,0
5247,-0.819322,1.599396,-0.314220,-0.427835,-0.019880,-1.206839,-1.078588,1.855066,0.446999,-0.209672,...,-0.534225,-1.382148,1.922498,-0.056317,1.248223,0.521329,-0.705488,0.367373,1.532914,1
5248,-0.929713,1.757020,-0.275950,-0.868771,-0.470645,1.814626,-0.040183,0.938170,-0.250300,0.564872,...,-0.679699,-0.490952,-0.320869,0.175378,1.885228,0.448269,-0.743421,-1.025090,0.029637,1


In [ ]:
test_df_scaled

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,-4.062163,0.869420,-0.768300,-1.130934,-1.774641,0.099667,1.415031,1.779542,-0.222279,-0.402718,...,-1.268801,-0.244584,-0.363536,-0.901544,0.342039,-0.319875,0.011572,-0.798998,-0.325406,-1.883039
1,-0.949823,0.962976,0.693248,0.053444,0.434113,-0.305150,-0.948858,0.887436,-0.082573,-0.271928,...,-0.040625,-1.285441,1.078295,-0.052454,-0.717109,-1.080296,1.160557,1.746030,0.901417,-0.478929
2,0.799706,-0.694988,2.297847,0.801373,-2.062973,0.674021,-0.321223,0.148739,-1.319361,-0.370337,...,0.794017,-0.918793,-0.599865,-0.695799,-0.706791,0.218305,-0.381484,-0.957490,-0.673937,1.218565
3,-0.359747,1.846766,0.375630,1.513088,-0.434720,1.176530,0.088423,0.500691,-0.892642,0.385583,...,-0.913837,-2.266712,-0.891630,-0.519032,0.019113,-2.498487,-2.148211,0.739530,-0.241893,-0.493195
4,1.117084,-1.190756,-2.074982,0.835166,0.982050,0.385633,-0.124357,-0.320628,0.899528,-0.208454,...,-3.232167,-2.651627,0.646511,0.086629,0.451416,-1.471778,0.879319,-0.989634,1.739753,-0.221348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,0.542995,-0.451495,0.549378,0.442481,0.808844,0.169974,1.048984,-0.922067,1.055320,-0.035273,...,0.501468,-1.727959,-0.190028,-1.129183,-0.732838,-0.238598,-0.832237,0.533913,1.115070,-0.677542
2246,0.667700,-0.168567,0.873229,0.799528,1.105208,0.614897,0.885530,-0.784996,-1.120739,1.003935,...,0.792147,-0.820967,-1.067825,-0.902190,0.119690,-0.440662,-0.249442,0.837870,-1.122689,0.046153
2247,0.932880,1.264657,-0.168397,-1.453847,0.151127,-0.197219,-0.982791,1.448526,-1.169106,0.612368,...,0.174091,-0.931052,0.749257,-0.977799,0.275431,-0.465143,1.850189,0.931684,-1.239539,1.563621
2248,0.707407,-0.240223,0.773201,0.660357,1.103867,0.596800,-1.220745,0.098200,-0.915558,0.860767,...,0.701845,-0.747803,-0.005669,0.196817,-0.810983,-0.207824,-0.036273,0.760524,-1.255091,0.515268


# **Feature reduction**

In [ ]:
pca = PCA(n_components=200)

In [ ]:
features = train_df_scaled.iloc[:, :-1]
labels = train_df_scaled['labels']
reduced_features = pca.fit_transform(features)
train_df_red = pd.DataFrame(reduced_features, columns=[f'f_{i}' for i in range(reduced_features.shape[1])])
train_df_red['labels'] = labels

In [ ]:
test_reduced_features = pca.transform(test_df_scaled)
test_df_red = pd.DataFrame(test_reduced_features, columns=[f'f_{i}' for i in range(test_reduced_features.shape[1])])

In [ ]:
train_df_red

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_191,f_192,f_193,f_194,f_195,f_196,f_197,f_198,f_199,labels
0,6.954544,-3.062235,-2.016013,0.225399,0.096753,-0.112102,0.300183,0.763386,1.207449,0.085996,...,-1.355272,0.804554,0.091084,-1.315363,2.147075,0.088478,1.969311,0.701653,0.860030,0
1,9.524688,-3.560576,0.270055,-1.181192,-1.045177,-0.222822,2.440751,0.787334,-1.730253,-0.081714,...,-1.819736,0.872865,3.244393,1.175344,0.294924,-0.521864,-0.170796,0.859821,3.135918,1
2,8.462361,-2.799490,-1.484716,1.200309,0.793290,-0.313780,2.099393,0.651638,-0.504515,1.783616,...,-2.240149,0.697564,-0.945519,1.352069,0.848925,-1.567326,-0.755854,-0.264101,-2.009253,1
3,9.024684,-4.319133,-1.992020,-0.578075,1.148346,-1.603091,0.003767,-0.502559,0.319303,-0.636368,...,0.433201,1.905069,0.605633,-0.104279,0.147855,-0.718708,-1.076443,-2.700990,-0.210652,0
4,8.280571,-2.747985,-2.031891,-0.334265,1.736726,1.386421,1.274329,-0.146759,1.205839,-0.773560,...,-2.047316,-0.545835,-1.111499,-1.141047,-0.439287,-1.713400,0.105189,-1.312963,0.309197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245,-15.819361,0.950299,-0.455001,-1.971675,-2.236437,8.766446,10.812448,-12.937847,-3.150831,0.662098,...,0.140368,-0.387708,0.043026,0.424473,-0.336857,0.202340,-0.250033,-0.023570,0.069766,0
5246,-15.479482,16.997910,-4.137286,2.998927,1.675495,6.358802,-8.409569,-0.380268,-2.207330,-1.110060,...,-0.058812,-0.255945,-0.136420,0.397605,-0.461175,0.244933,-0.229232,-0.171352,0.270051,0
5247,8.541983,15.755060,5.278474,-18.510527,8.776877,-4.246212,2.468840,-0.412708,-0.683358,-0.559534,...,0.075849,0.416394,-0.042423,-1.081067,0.237125,0.122489,-0.836718,0.321420,0.464564,1
5248,9.211937,18.645144,11.171351,13.627334,-9.829741,-4.621547,-4.586131,-3.422143,-0.918160,4.678789,...,-0.105767,-0.104150,0.627831,0.302340,0.437004,-0.596878,0.125160,0.264223,-0.043951,1


In [ ]:
test_df_red

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_190,f_191,f_192,f_193,f_194,f_195,f_196,f_197,f_198,f_199
0,9.202193,1.997424,-1.066525,-0.653316,0.603734,-1.768011,-1.061731,-1.432810,0.984461,-0.704494,...,0.229950,0.306079,0.431741,-0.350001,1.360465,0.542328,-0.448099,-1.200175,0.304863,-0.564976
1,5.722886,-1.855478,1.050528,0.809669,3.651287,1.199354,0.900517,0.507536,-0.959598,0.430609,...,0.087852,0.450092,1.820563,1.927420,-0.524680,0.470356,-0.466424,1.029102,0.502624,0.198588
2,5.710969,0.575912,-0.286213,0.073803,0.280394,-0.106998,1.153743,-2.612667,2.104617,3.463203,...,1.168002,0.642616,0.658994,-0.880165,-0.734558,1.018894,-0.253874,-0.558707,0.541215,0.376310
3,5.365582,-1.005402,-0.937581,1.777703,1.488134,0.385129,-0.040966,1.122046,-2.147851,1.370222,...,-0.862163,-0.293309,-2.116522,-2.862875,0.009413,-1.084880,0.848707,1.008399,1.005295,0.197369
4,4.397665,-2.452061,-2.085852,2.394450,1.152894,-1.228275,2.088818,-0.168980,1.610901,1.023808,...,-1.566521,-0.113503,-0.285970,-0.908441,2.582582,-0.811544,-1.898331,-0.393258,0.554457,1.626130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,-15.084843,-13.076410,7.849938,2.875500,8.750536,-1.517316,-3.978401,10.591386,-1.083012,1.061440,...,-0.939312,-0.045381,0.452449,-0.214584,-0.093042,-0.422658,-0.131575,0.339303,0.650970,-0.707149
2246,-20.074608,-1.598502,5.730116,5.809555,-8.521463,1.546723,2.878178,-6.835159,1.354899,-0.643549,...,-0.361808,0.159118,0.441770,0.232483,0.518129,-0.273166,-0.504141,-0.231733,0.338604,-0.967081
2247,-0.393363,15.123061,0.882806,-9.102697,-5.789218,-6.234466,-4.989054,6.669637,3.398416,2.187647,...,-0.122318,0.832574,0.716883,0.650444,-0.256002,0.614031,-0.153663,0.524322,-0.509734,0.258749
2248,-20.090358,6.067893,3.646399,-6.092793,-0.498236,-1.012095,5.248083,-6.293543,-8.089582,0.773316,...,-0.016867,0.097551,-0.363576,0.116396,-0.034000,-0.099638,-0.513411,-0.106391,-0.182620,0.137168


# **Adaboost classifier**

In [ ]:
X = train_df_red.iloc[:, :-1]
y = train_df_red['labels']

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

base_classifier = DecisionTreeClassifier(max_depth=3)
n_estimators = 50
learning_rate = 1.0
ada_boost = AdaBoostClassifier(estimator=base_classifier,
                               n_estimators=n_estimators,
                               learning_rate=learning_rate)

ada_boost.fit(X_train, y_train)

y_pred = ada_boost.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8457142857142858


In [ ]:
output1 = ada_boost.predict(test_df_red)

In [ ]:
print(output1)

[1 0 0 ... 1 0 1]


In [ ]:
output1 = pd.DataFrame({'id': range(1, len(output1) + 1), 'labels': output1})

In [ ]:
print(output1)

        id  labels
0        1       1
1        2       0
2        3       0
3        4       0
4        5       0
...    ...     ...
2245  2246       0
2246  2247       0
2247  2248       1
2248  2249       0
2249  2250       1

[2250 rows x 2 columns]


In [ ]:
output1.to_csv('output1.csv', index=False)

# **Creating Dataloaders**

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels
    transforms.ToTensor(),
])

In [ ]:
kwargs = {'num_workers': 2, 'pin_memory': True}

In [ ]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, data_df, transform=None):
        self.data = data_df.values[:, :-1].astype(float)
        self.labels = data_df['labels'].values.astype(int)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        features = torch.tensor(self.data[index], dtype=torch.float32)
        label = torch.tensor(self.labels[index], dtype=torch.long)

        # Reshape the features tensor to (C, H, W) if it's 1-dimensional
        if features.dim() == 1:
            features = features.unsqueeze(0)

        if self.transform:
            features = self.transform(features)

        return features, label


trainset = TrainDataset(train_df_red, transform=transform)

batch_size = 64
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True,  **kwargs)

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, data_df, transform=None):
        self.data = data_df.values.astype(float)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        features = torch.tensor(self.data[index], dtype=torch.float32)

        # Reshaping the features tensor to (C, H, W) if it's 1-dimensional
        if features.dim() == 1:
            features = features.unsqueeze(0)

        if self.transform:
            features = self.transform(features)

        return features


testset = TestDataset(test_df_red,transform=transform)
testloader = DataLoader(testset, batch_size=batch_size,shuffle=False,  **kwargs)

# **Training and Predicting Functions**

In [ ]:
def loss_function(pred, lb):
    loss = nn.CrossEntropyLoss()(pred, lb)
    return loss

def train(model, epoch, train_losses, train_accs):
    model.train()
    train_loss = 0
    cor = 0
    total = 0
    for batch_idx, (data, label) in enumerate(trainloader):
        data = data.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = loss_function(pred, label)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        _, pred_lb = pred.max(dim=1)
        cor += (pred_lb == label).sum().to(device)
        total += len(data)
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader),
                loss.item() / len(data)))
    train_acc = cor * 100 / total
    train_losses.append(train_loss / len(trainloader.dataset))
    train_accs.append(train_acc)
    print('====> Epoch: {} Average loss: {:.4f} acc: {:.4f}'.format(
        epoch, train_losses[-1], train_accs[-1]))


In [ ]:
def test_model(model, testloader):
    model.eval()  # Set model to evaluation mode
    predictions = []
    with torch.no_grad():
        for i, data in enumerate(testloader):
            inputs = data.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())

    return predictions

def predict(model, testloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for data in testloader:
            data = data.to(device)
            pred = model(data)
            _, pred_labels = pred.max(dim=1)
            predictions.extend(pred_labels.tolist())
    return predictions


In [ ]:
# List alL the available models
import torchvision.models as models

model_names = sorted(name for name in models.__dict__ if name.islower() and not name.startswith("__"))

for name in model_names:
    print(name)

_api
_meta
_utils
alexnet
convnext
convnext_base
convnext_large
convnext_small
convnext_tiny
densenet
densenet121
densenet161
densenet169
densenet201
detection
efficientnet
efficientnet_b0
efficientnet_b1
efficientnet_b2
efficientnet_b3
efficientnet_b4
efficientnet_b5
efficientnet_b6
efficientnet_b7
efficientnet_v2_l
efficientnet_v2_m
efficientnet_v2_s
get_model
get_model_builder
get_model_weights
get_weight
googlenet
inception
inception_v3
list_models
maxvit
maxvit_t
mnasnet
mnasnet0_5
mnasnet0_75
mnasnet1_0
mnasnet1_3
mobilenet
mobilenet_v2
mobilenet_v3_large
mobilenet_v3_small
mobilenetv2
mobilenetv3
optical_flow
quantization
regnet
regnet_x_16gf
regnet_x_1_6gf
regnet_x_32gf
regnet_x_3_2gf
regnet_x_400mf
regnet_x_800mf
regnet_x_8gf
regnet_y_128gf
regnet_y_16gf
regnet_y_1_6gf
regnet_y_32gf
regnet_y_3_2gf
regnet_y_400mf
regnet_y_800mf
regnet_y_8gf
resnet
resnet101
resnet152
resnet18
resnet34
resnet50
resnext101_32x8d
resnext101_64x4d
resnext50_32x4d
segmentation
shufflenet_v2_x0_5
shu

# **Resnet18 Model**

In [ ]:
from torchvision.models import resnet18,ResNet18_Weights

In [ ]:
#loading the ResNet18 model and modifying the last fully connected layer to have 2 output nodes instead of 1000
model1 = resnet18(weights=None)#ResNet18_Weights.DEFAULT)

num_ftrs = model1.fc.in_features
model1.fc = nn.Linear(num_ftrs, 2)

In [ ]:
# Loading the AdamW optimizer
optimizer = optim.AdamW(model1.parameters(), lr=0.001)

In [ ]:
model= model1.to(device)

In [ ]:
train_losses = []
train_accs = []

for epoch in range(1,3): # change the epochs in the final run
    train(model1, epoch, train_losses, train_accs)

Train Epoch: 1 [0/5250 (0%)]	Loss: 0.016290


KeyboardInterrupt: ignored

In [ ]:
predictions1 = predict(model1, testloader)

In [ ]:
predictions1

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
output1 = pd.DataFrame({'id': range(1, len(predictions1) + 1), 'labels': predictions1})

In [ ]:
output1

,id,labels
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
2245,2246,0
2246,2247,0
2247,2248,0
2248,2249,0


In [ ]:
output1.to_csv('output1.csv', index=False)

In [ ]:
plt.plot(train_losses, label='Training Loss')
plt.title('Model Training Loss ')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_accs = torch.tensor(train_accs)

In [ ]:
plt.plot(train_accs, label='Training Accuracy')
plt.title('Model Training Accuracy ')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# **Resnet50 Model**

In [ ]:
from torchvision.models import resnet50,ResNet50_Weights


In [ ]:
model2 = resnet50(weights=ResNet50_Weights.DEFAULT)

num_ftrs = model2.fc.in_features
model2.fc = nn.Linear(num_ftrs, 2)

In [ ]:
optimizer = optim.Adam(model2.parameters(), lr=0.01)

In [ ]:
model2= model2.to(device)

In [ ]:
train_losses = []
train_accs = []

for epoch in range(1,11):
    train(model2, epoch, train_losses, train_accs)

In [ ]:
predictions2 = test_model(model2, testloader)


In [ ]:
output2 = pd.DataFrame({'id': range(1, len(predictions2) + 1), 'labels': predictions2})

In [ ]:
output2

In [ ]:
output2.to_csv('output2.csv', index=False)

In [ ]:
plt.plot(train_losses, label='Training Loss')
plt.title('Model Training Loss ')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_accs = torch.tensor(train_accs)

In [ ]:
plt.plot(train_accs, label='Training Accuracy')
plt.title('Model Training Accuracy ')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# **VGG13 Model**

In [ ]:
from torchvision.models import vgg13,VGG13_Weights


In [ ]:
model3 = vgg13(weights=VGG13_Weights.DEFAULT)

# Modify the output layer for binary classification
num_features = model3.classifier[6].in_features
model3.classifier[6] = nn.Linear(num_features, 2)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:10<00:00, 54.9MB/s]


In [ ]:
optimizer = optim.Adam(model3.parameters(), lr=0.01)

In [ ]:
model3= model3.to(device)

In [ ]:
train_losses = []
train_accs = []

for epoch in range(1,11):
    train(model3, epoch, train_losses, train_accs)

In [ ]:
predictions3 = test_model(model3, testloader)


In [ ]:
output3 = pd.DataFrame({'id': range(1, len(predictions3) + 1), 'labels': predictions3})

In [ ]:
output3

In [ ]:
output3.to_csv('output3.csv', index=False)

In [ ]:
plt.plot(train_losses, label='Training Loss')
plt.title('Model Training Loss ')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_accs = torch.tensor(train_accs)

In [ ]:
plt.plot(train_accs, label='Training Accuracy')
plt.title('Model Training Accuracy ')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# **ConvNext Model**

In [ ]:
from torchvision.models import convnext_small,ConvNeXt_Small_Weights

In [ ]:
model4 = convnext_small(weights=ConvNeXt_Small_Weights.DEFAULT)

In [ ]:
optimizer = optim.Adam(model4.parameters(), lr=0.01)

In [ ]:
model4= model4.to(device)

In [ ]:
train_losses = []
train_accs = []

for epoch in range(1,11):
    train(model4, epoch, train_losses, train_accs)

In [ ]:
predictions4 = test_model(model4, testloader)


In [ ]:
output4 = pd.DataFrame({'id': range(1, len(predictions4) + 1), 'labels': predictions4})

In [ ]:
output4

In [ ]:
output4.to_csv('output4.csv', index=False)

In [ ]:
plt.plot(train_losses, label='Training Loss')
plt.title('Model Training Loss ')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_accs = torch.tensor(train_accs)

In [ ]:
plt.plot(train_accs, label='Training Accuracy')
plt.title('Model Training Accuracy ')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# **Vision Transformers**

In [ ]:
from torchvision.models import vit_b_32,ViT_B_32_Weights

In [ ]:
model5 = vit_b_32(weights=ViT_B_32_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/vit_b_32-d86f8d99.pth" to /root/.cache/torch/hub/checkpoints/vit_b_32-d86f8d99.pth
100%|██████████| 337M/337M [00:06<00:00, 56.3MB/s]


In [ ]:
optimizer = optim.Adam(model5.parameters(), lr=0.01)

In [ ]:
model5= model5.to(device)

In [ ]:
train_losses = []
train_accs = []

for epoch in range(1,11):
    train(model5, epoch, train_losses, train_accs)

Train Epoch: 1 [0/5250 (0%)]	Loss: 0.137560
====> Epoch: 1 Average loss: 0.0146 acc: 67.3714
Train Epoch: 2 [0/5250 (0%)]	Loss: 0.022372
====> Epoch: 2 Average loss: 0.0105 acc: 68.4952
Train Epoch: 3 [0/5250 (0%)]	Loss: 0.020859
====> Epoch: 3 Average loss: 0.0106 acc: 68.9143
Train Epoch: 4 [0/5250 (0%)]	Loss: 0.010979
====> Epoch: 4 Average loss: 0.0093 acc: 71.6381
Train Epoch: 5 [0/5250 (0%)]	Loss: 0.011587
====> Epoch: 5 Average loss: 0.0094 acc: 73.3333
Train Epoch: 6 [0/5250 (0%)]	Loss: 0.010298
====> Epoch: 6 Average loss: 0.0094 acc: 73.3333
Train Epoch: 7 [0/5250 (0%)]	Loss: 0.009296
====> Epoch: 7 Average loss: 0.0093 acc: 73.3333
Train Epoch: 8 [0/5250 (0%)]	Loss: 0.010525
====> Epoch: 8 Average loss: 0.0094 acc: 73.3333
Train Epoch: 9 [0/5250 (0%)]	Loss: 0.008626


KeyboardInterrupt: ignored

In [ ]:
predictions5 = test_model(model5, testloader)

In [ ]:
output5 = pd.DataFrame({'id': range(1, len(predictions5) + 1), 'labels': predictions5})

In [ ]:
output5

,id,labels
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
2245,2246,0
2246,2247,0
2247,2248,0
2248,2249,0


In [ ]:
output5.to_csv('output5.csv', index=False)

In [ ]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
plt.plot(train_losses, label='Training Loss')
plt.title('Model Training Loss ')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_accs = torch.tensor(train_accs)

In [ ]:
plt.plot(train_accs, label='Training Accuracy')
plt.title('Model Training Accuracy ')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()